## Revolut FinCrime Challenge: Model Building

There will be a notebook for each one of the Machine Learning Pipeline steps:

1. Data Analysis
2. Feature Engineering
3. Feature Selection
4. Model Building

**This is the notebook for step 4: Model Building**


## Predicting Fradulent transactions

The aim of the project is to build a machine learning model to find the fraudsters and take appropriate actions.

![SegmentLocal](fraud.gif "segment")

### Why is this important? 

Fraudsters can use our App to steal the data from other people's money from the outside into an account via TOP-UP's. So finding them and blocking them is very necessary.

## Machine Learning Model build

In the following cells, I will finally build the machine learning model, utilising the engineered data and the pre-selected features. 


Let's go ahead and load the dataset.

In [1]:
# to handle datasets
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 80
import seaborn as sns
%matplotlib inline

# to build the models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# to persist the model
from sklearn.externals import joblib

# to evaluate the models
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import precision_recall_curve
from math import sqrt

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
# load dataset
# We load the datasets with the engineered values: we built and saved this datasets in the previous notebook.

train = pd.read_csv('../data/xtrain.csv')
test = pd.read_csv('../data/xtest.csv')

train.head()

,user_id,transaction_id,transaction_ts,registered_ts,birth_date,transaction_date,user_age,is_fraud,avg_transaction_count,amount_gbp,transaction_hr,hr_gt_nine,state_COMPLETED,state_DECLINED,state_FAILED,state_REVERTED,currency_BGN,currency_CHF,currency_CZK,currency_EUR,currency_GBP,currency_HRK,currency_HUF,currency_PLN,currency_RON,currency_Rare,currency_SEK,currency_USD,country_BG,country_CH,country_CZ,country_DE,country_ES,country_FR,country_GB,country_HR,country_HU,country_IE,country_IT,country_LT,country_LV,country_PL,country_PT,country_RO,country_Rare,country_SE,type_ATM,type_CARD_PAYMENT,type_EXCHANGE,type_FEE,type_TOPUP,type_TRANSFER,age_18-27,age_28-37,age_38-47,age_48-57,age_58-67,age_Rare
0,00001f33-1d47-47a3-8955-e719172e788b,46285eca-e1b1-43a2-b8b0-c173b48f5a95,2019-04-21 11:04:53.081,2019-04-11 20:49:42.623,1994-08-17 00:00:00.000,2019-04-21,25,0,-0.508167,0.054028,-0.580791,0.504333,0.727990,-0.250637,-0.331993,-0.481188,-0.112081,-0.174942,-0.138751,1.256661,-0.527487,-0.13125,-0.176167,-0.322369,-0.265881,-0.197194,-0.1061,-0.290565,-0.116779,-0.199412,-0.151228,-0.121195,-0.233889,-0.316005,-0.616132,-0.107685,-0.176442,3.487115,-0.133202,-0.122211,-0.115657,-0.386638,-0.211368,-0.290697,-0.272798,-0.10378,-0.136299,1.620741,-0.226587,-0.251627,-1.136748,-0.187375,1.370363,-0.639831,-0.482421,-0.347228,-0.200353,-0.166816
1,00001f33-1d47-47a3-8955-e719172e788b,1be7d138-fbaf-4216-8dc8-5056ec5f972c,2019-04-21 10:14:53.814,2019-04-11 20:49:42.623,1994-08-17 00:00:00.000,2019-04-21,25,0,-0.508167,0.043652,-0.778896,0.504333,0.727990,-0.250637,-0.331993,-0.481188,-0.112081,-0.174942,-0.138751,1.256661,-0.527487,-0.13125,-0.176167,-0.322369,-0.265881,-0.197194,-0.1061,-0.290565,-0.116779,-0.199412,-0.151228,-0.121195,-0.233889,-0.316005,-0.616132,-0.107685,-0.176442,3.487115,-0.133202,-0.122211,-0.115657,-0.386638,-0.211368,-0.290697,-0.272798,-0.10378,-0.136299,1.620741,-0.226587,-0.251627,-1.136748,-0.187375,1.370363,-0.639831,-0.482421,-0.347228,-0.200353,-0.166816
2,00001f33-1d47-47a3-8955-e719172e788b,a9aa681d-451e-44c5-8df0-687661ac583d,2019-04-19 21:05:53.192,2019-04-11 20:49:42.623,1994-08-17 00:00:00.000,2019-04-19,25,0,-0.508167,0.346044,1.400259,0.504333,0.727990,-0.250637,-0.331993,-0.481188,-0.112081,-0.174942,-0.138751,1.256661,-0.527487,-0.13125,-0.176167,-0.322369,-0.265881,-0.197194,-0.1061,-0.290565,-0.116779,-0.199412,-0.151228,-0.121195,-0.233889,-0.316005,-0.616132,-0.107685,-0.176442,3.487115,-0.133202,-0.122211,-0.115657,-0.386638,-0.211368,-0.290697,-0.272798,-0.10378,-0.136299,-0.617002,-0.226587,-0.251627,0.879702,-0.187375,1.370363,-0.639831,-0.482421,-0.347228,-0.200353,-0.166816
3,00001f33-1d47-47a3-8955-e719172e788b,e6021128-f4c1-4164-b3de-697e66ad613c,2019-04-11 20:55:20.996,2019-04-11 20:49:42.623,1994-08-17 00:00:00.000,2019-04-11,25,0,-0.508167,-0.121944,1.202154,0.504333,0.727990,-0.250637,-0.331993,-0.481188,-0.112081,-0.174942,-0.138751,1.256661,-0.527487,-0.13125,-0.176167,-0.322369,-0.265881,-0.197194,-0.1061,-0.290565,-0.116779,-0.199412,-0.151228,-0.121195,-0.233889,-0.316005,-0.616132,-0.107685,-0.176442,3.487115,-0.133202,-0.122211,-0.115657,-0.386638,-0.211368,-0.290697,-0.272798,-0.10378,-0.136299,-0.617002,-0.226587,3.974133,-1.136748,-0.187375,1.370363,-0.639831,-0.482421,-0.347228,-0.200353,-0.166816
4,00001f33-1d47-47a3-8955-e719172e788b,9499c9c9-c9a9-410f-820d-c6e92fed27fb,2019-04-11 20:53:54.700,2019-04-11 20:49:42.623,1994-08-17 00:00:00.000,2019-04-11,25,0,-0.508167,-0.144518,1.202154,0.504333,-1.373644,-0.250637,-0.331993,2.078191,-0.112081,-0.174942,-0.138751,1.256661,-0.527487,-0.13125,-0.176167,-0.322369,-0.265881,-0.197194,-0.1061,-0.290565,-0.116779,-0.199412,-0.151228,-0.121195,-0.233889,-0.316005,-0.616132,-0.107685,-0.176442,3.487115,-0.133202,-0.122211,-0.115657,-0.386638,-0.211368,-0.290697,-0.272798,-0.10378,-0.136299,-0.617002,-0.226587,-0.251627,0.879702,-0.187375,1.370363,-0.639831,-0.482421,-0.347228,-0.200353,-0.166816


In [3]:
test.query('is_fraud == 1')

,user_id,transaction_id,transaction_ts,registered_ts,birth_date,transaction_date,user_age,is_fraud,avg_transaction_count,amount_gbp,transaction_hr,hr_gt_nine,state_COMPLETED,state_DECLINED,state_FAILED,state_REVERTED,currency_BGN,currency_CHF,currency_CZK,currency_EUR,currency_GBP,currency_HRK,currency_HUF,currency_PLN,currency_RON,currency_Rare,currency_SEK,currency_USD,country_BG,country_CH,country_CZ,country_DE,country_ES,country_FR,country_GB,country_HR,country_HU,country_IE,country_IT,country_LT,country_LV,country_PL,country_PT,country_RO,country_Rare,country_SE,type_ATM,type_CARD_PAYMENT,type_EXCHANGE,type_FEE,type_TOPUP,type_TRANSFER,age_18-27,age_28-37,age_38-47,age_48-57,age_58-67,age_Rare
1711,0211c232-1cc5-4591-9d1e-e132dd89ce92,abcd6a12-3465-4ac6-9d84-f4df4e52bada,2019-04-06 12:25:43.288,2019-04-06 12:22:34.392,1979-10-14 00:00:00.000,2019-04-06,40,1,2.249513,-0.144518,-0.382686,0.504333,-1.373644,-0.250637,-0.331993,2.078191,-0.112081,-0.174942,-0.138751,-0.795759,1.895780,-0.13125,-0.176167,-0.322369,-0.265881,-0.197194,-0.1061,-0.290565,-0.116779,-0.199412,-0.151228,-0.121195,-0.233889,-0.316005,1.623028,-0.107685,-0.176442,-0.28677,-0.133202,-0.122211,-0.115657,-0.386638,-0.211368,-0.290697,-0.272798,-0.10378,-0.136299,-0.617002,-0.226587,-0.251627,0.879702,-0.187375,-0.729734,-0.639831,2.072877,-0.347228,-0.200353,-0.166816
1712,0211c232-1cc5-4591-9d1e-e132dd89ce92,961a3a74-3c05-4353-83f4-042886ba9eb7,2019-04-06 21:56:54.959,2019-04-06 12:22:34.392,1979-10-14 00:00:00.000,2019-04-06,40,1,2.249513,0.278958,1.400259,0.504333,0.727990,-0.250637,-0.331993,-0.481188,-0.112081,-0.174942,-0.138751,-0.795759,1.895780,-0.13125,-0.176167,-0.322369,-0.265881,-0.197194,-0.1061,-0.290565,-0.116779,-0.199412,-0.151228,-0.121195,-0.233889,-0.316005,1.623028,-0.107685,-0.176442,-0.28677,-0.133202,-0.122211,-0.115657,-0.386638,-0.211368,-0.290697,-0.272798,-0.10378,-0.136299,-0.617002,-0.226587,-0.251627,0.879702,-0.187375,-0.729734,-0.639831,2.072877,-0.347228,-0.200353,-0.166816
1713,0211c232-1cc5-4591-9d1e-e132dd89ce92,d43859ab-c81d-46ce-802b-2588ac32ea84,2019-04-06 12:34:02.339,2019-04-06 12:22:34.392,1979-10-14 00:00:00.000,2019-04-06,40,1,2.249513,0.278958,-0.382686,0.504333,0.727990,-0.250637,-0.331993,-0.481188,-0.112081,-0.174942,-0.138751,-0.795759,1.895780,-0.13125,-0.176167,-0.322369,-0.265881,-0.197194,-0.1061,-0.290565,-0.116779,-0.199412,-0.151228,-0.121195,-0.233889,-0.316005,1.623028,-0.107685,-0.176442,-0.28677,-0.133202,-0.122211,-0.115657,-0.386638,-0.211368,-0.290697,-0.272798,-0.10378,-0.136299,-0.617002,-0.226587,-0.251627,0.879702,-0.187375,-0.729734,-0.639831,2.072877,-0.347228,-0.200353,-0.166816
1714,0211c232-1cc5-4591-9d1e-e132dd89ce92,279af71c-c3bb-4ad2-ac90-214c27de1871,2019-04-06 22:01:38.707,2019-04-06 12:22:34.392,1979-10-14 00:00:00.000,2019-04-06,40,1,2.249513,0.283194,1.598364,0.504333,0.727990,-0.250637,-0.331993,-0.481188,-0.112081,-0.174942,-0.138751,-0.795759,1.895780,-0.13125,-0.176167,-0.322369,-0.265881,-0.197194,-0.1061,-0.290565,-0.116779,-0.199412,-0.151228,-0.121195,-0.233889,-0.316005,1.623028,-0.107685,-0.176442,-0.28677,-0.133202,-0.122211,-0.115657,-0.386638,-0.211368,-0.290697,-0.272798,-0.10378,7.336815,-0.617002,-0.226587,-0.251627,-1.136748,-0.187375,-0.729734,-0.639831,2.072877,-0.347228,-0.200353,-0.166816
1715,0211c232-1cc5-4591-9d1e-e132dd89ce92,bdee2b4d-ff84-46b1-85cd-e49d84d85661,2019-04-06 12:26:02.314,2019-04-06 12:22:34.392,1979-10-14 00:00:00.000,2019-04-06,40,1,2.249513,0.278958,-0.382686,0.504333,0.727990,-0.250637,-0.331993,-0.481188,-0.112081,-0.174942,-0.138751,-0.795759,1.895780,-0.13125,-0.176167,-0.322369,-0.265881,-0.197194,-0.1061,-0.290565,-0.116779,-0.199412,-0.151228,-0.121195,-0.233889,-0.316005,1.623028,-0.107685,-0.176442,-0.28677,-0.133202,-0.122211,-0.115657,-0.386638,-0.211368,-0.290697,-0.272798,-0.10378,-0.136299,-0.617002,-0.226587,-0.251627,0.879702,-0.187375,-0.729734,-0.639831,2.072877,-0.347228,-0.200353,-0.166816
...,...,...,...,...,...

In [4]:
single_test_input = test[0:1]

In [5]:
test.query('transaction_id == "8b46f81c-a602-4d25-b81b-9d3b959f26ae"')

,user_id,transaction_id,transaction_ts,registered_ts,birth_date,transaction_date,user_age,is_fraud,avg_transaction_count,amount_gbp,transaction_hr,hr_gt_nine,state_COMPLETED,state_DECLINED,state_FAILED,state_REVERTED,currency_BGN,currency_CHF,currency_CZK,currency_EUR,currency_GBP,currency_HRK,currency_HUF,currency_PLN,currency_RON,currency_Rare,currency_SEK,currency_USD,country_BG,country_CH,country_CZ,country_DE,country_ES,country_FR,country_GB,country_HR,country_HU,country_IE,country_IT,country_LT,country_LV,country_PL,country_PT,country_RO,country_Rare,country_SE,type_ATM,type_CARD_PAYMENT,type_EXCHANGE,type_FEE,type_TOPUP,type_TRANSFER,age_18-27,age_28-37,age_38-47,age_48-57,age_58-67,age_Rare
0,0001bd0a-3654-4e19-9d63-2df27e11ebd3,8b46f81c-a602-4d25-b81b-9d3b959f26ae,2019-04-05 18:27:43.950,2019-04-05 08:01:32.725,1956-05-25 00:00:00.000,2019-04-05,63,0,0.594905,-0.114956,0.805944,0.504333,0.72799,-0.250637,-0.331993,-0.481188,-0.112081,-0.174942,-0.138751,-0.795759,1.89578,-0.13125,-0.176167,-0.322369,-0.265881,-0.197194,-0.1061,-0.290565,-0.116779,-0.199412,-0.151228,-0.121195,-0.233889,-0.316005,1.623028,-0.107685,-0.176442,-0.28677,-0.133202,-0.122211,-0.115657,-0.386638,-0.211368,-0.290697,-0.272798,-0.10378,-0.136299,-0.617002,-0.226587,3.974133,-1.136748,-0.187375,-0.729734,-0.639831,-0.482421,-0.347228,4.991197,-0.166816


In [6]:
r[0]

NameError: name 'r' is not defined

In [ ]:
# capture the target
y_train = train['is_fraud']
y_test = test['is_fraud']

In [ ]:
# load selected features
# we selected the features in the previous notebook

features = pd.read_csv('../data/selected_features.csv', header=None)
features = [x for x in features[0]]

In [ ]:
features

In [ ]:
# reduce the train and test set to the desired features

X_train = train[features]
X_test = test[features]

In [ ]:
# lets check the target class distribution

y_train.value_counts()/len(y_train) * 100

From the above transactions we can observe that the dataset is clearly imbalanced, as there are only 0.77% fradulent transactions. If we directly build the model on this data model can be biased towards the majority class. So,we need to handle this data properly.

### Handling the imbalanced Data

There are various ways to handle the imbalanced datasets.

1. Generate synthetic samples(Oversampling, SMOTE etc.)
2. Trying penalized models
3. Trying a different perspective(By looking at this problem as anomaly detection use case)
4. Assigning higher weights to minority class(using class_weight parameter)

For this problem I have choosed the last way to handle the imbalance data i.e by using class_weight parameter. However, for better results we have to try out various approaches to figure out the best.


### Logistic regression

Remember to set the seed.

In [ ]:
# train the model
log_model = LogisticRegression(random_state=0,class_weight={0:0.2,1:0.80}) # remember to set the random_state / seed
log_model.fit(X_train, y_train)
y_pred = log_model.predict(X_test)
y_pred_prob = log_model.predict_proba(X_test)[:,1]

In [ ]:
print(classification_report(y_test,y_pred))

Usually for imbalanced datasets we don't choose accuracy as the evaluation metric as it's value would be always high. So we need to lookup for other metrics like Precision, Recall, F1-score, AUC etc.

First we need to understand the business to understand which among False Positive, False negative is more costly.

For the True Positive, the model predicts as fraud user given he/she is really a fraud, we lock the user correctly, the company will have no costs or just administration fees.

For the True Negative, the model predicts it’s a non fradulent user given he/she is really good user, we will approve it and no costs occur.

For the False Positive, the model predicts as fraud user given he/she is good user, we will need to pay for the customer service fee( and potential we might loose a customer).

For the False Negative, the model predicts as good user given he/she is a fraud, we approve the order wrongly and need to pay for the chargebacks while losing the amount of the order.

Assuming we do have limited human resources for checking fradulent transactions, precision would be more important for us than the recall. Hence we can plot the precision-recall curve to decide upon the best threshold.


In [ ]:
def plot_precision_recall_curve(y_test,y_prob):
    precision, recall, thresholds = precision_recall_curve(y_test, y_prob) 
    pr_auc = metrics.auc(recall, precision)
    plt.title("Precision-Recall vs Threshold Chart")
    plt.plot(thresholds, precision[: -1], "b--", label="Precision")
    plt.plot(thresholds, recall[: -1], "r--", label="Recall")
    plt.ylabel("Precision, Recall")
    plt.xlabel("Threshold")
    plt.legend(loc="lower left")
    plt.ylim([0,1])

In [ ]:
plot_precision_recall_curve(y_test,y_pred_prob)

### Random Forest 

In [ ]:
# training the model
rf_model = RandomForestClassifier(n_estimators=125,random_state=0)
rf_model.fit(X_train, y_train)
y_pred_prob_rf = rf_model.predict_proba(X_test)[:,1]

In [ ]:
#y_pred = rf_model.predict(X_test)

In [ ]:
plot_precision_recall_curve(y_test,y_pred_prob_rf)

Ideally we want high precision and recall.For this model we got the trade off at threshold 0.25 with both precison and recall as 0.5. However as per my assumption I need a better precision than recall, I am choosing the threshold as 0.7, where precision is ~0.8 and recall is ~0.3 . 


Let's evaluate the performance of this strategy. As it is given in the task that performance should be at user level, let's join the training set with prediction to group the prediction output based upon the user.

In [ ]:
def calc_usr_lvl_metrics(data,prob_y,threshold):
    pred_y = (prob_y >= threshold).astype(int)
    data['pred']=pred_y
    temp_df = pd.DataFrame(data.groupby(['user_id','is_fraud'])['pred'].max())
    temp_df.reset_index(inplace=True)
    print(classification_report(temp_df['is_fraud'],temp_df['pred']))

In [ ]:
calc_usr_lvl_metrics(test,y_pred_prob_rf,0.7)

At user level, this threshold is not having high precision. Hence I increased the threshold to 0.85

In [ ]:
calc_usr_lvl_metrics(test,y_pred_prob_rf,0.85)

Finally, we end up getting a precision 0.66 and recall of 0.42 for the assumed strategy. However, this model can be further tuned to improve the performance.

In [ ]:
# Finally let's look at the feature importance

importance = pd.Series(np.abs(rf_model.feature_importances_))
importance.index = features
importance.sort_values(inplace=True, ascending=False)
importance.plot.bar(figsize=(18,6))
plt.title('Feature Importance')

We can see from the above plot that the newly created features like 'avg_transaction_count','transaction_hr' and 'age' variables have a significant importance in the model.

Finally, I have choosen the Random forest for building the models as it's performance is better than Logistic regression model . We can also try out other algorithms to find out the best algorithm. 

We can also perform hyper parameter tuning using Random search or grid search to increase the performance of this model.

### Improvements

If there is some more time I would have tried various other algorithms to pick the best one and also would have performed hyper parameter tuning using Random search or grid search to increase the performance of this model.

In [ ]:
# we persist the model for future use
joblib.dump(rf_model, '../artifacts/random_forest.pkl')

### References

https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc

https://stats.stackexchange.com/questions/384833/adjusting-probability-threshold-for-sklearns-logistic-regression-model

https://github.com/trainindata/deploying-machine-learning-models

https://medium.com/datadriveninvestor/rethinking-the-right-metrics-for-fraud-detection-4edfb629c423
